In [1]:
import numpy as np
from sklearn import preprocessing
import pandas as pd
pd.set_option('max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot')
import datetime
import lightgbm as lgb
from scipy import stats
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split, KFold
from wordcloud import WordCloud
from collections import Counter
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import StandardScaler
stop = set(stopwords.words('english'))
import os
#import xgboost as xgb
#import lightgbm as lgb
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib
import json
import ast
import eli5
from functools import reduce
import warnings
from sklearn.metrics import mean_squared_error
from scipy import stats
from math import sqrt
from lightgbm import plot_tree
from hyperopt import hp, tpe
from hyperopt.fmin import fmin
from hyperopt import Trials
from hyperopt import fmin
from hyperopt import STATUS_OK
from hyperopt.pyll.stochastic import sample
import time
import gc
import tensorflow as tf
from tensorflow.python.client import device_lib

device_lib.list_local_devices()
warnings.filterwarnings('ignore')


c:\users\chadchang\appdata\local\programs\python\python37\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
train = pd.read_csv('dataset-0510/train.csv')
test  = pd.read_csv('dataset-0510/test.csv')
data = pd.concat([train, test], axis=0)
gc.collect()
data.drop([i for i in data.columns if 'index' in i and np.sum(data[i]) == 70000], axis = 1, inplace = True)
data.drop(['village', 'doc_rate', 'master_rate', 'bachelor_rate', 'highschool_rate', 'jobschool_rate', 'junior_rate'], axis = 1, inplace = True)
train = data[:60000]
test = data[60000:]
#train.drop(['village'], axis =1, inplace = True)
#test.drop(['village'], axis =1, inplace = True)

In [3]:
#categorical feature to one-hot
def hit_score(preds, train_data):
    trues  = train_data.get_label()
    trues = np.expm1(trues)
    preds = np.expm1(preds)
    scores = (np.absolute(preds - trues) / trues) > 0.1
    hit_error = np.sum(scores) / train_data.num_data()
    return 'hit_error', round(scores), False

def one_hot(train, test, categorical_features):
    data = pd.concat([train, test], axis=0)
    for i in categorical_features:
        data = data.join(pd.get_dummies(data[i], prefix = i))
        data.drop(i, axis = 1, inplace =True)
    train = data[:60000]
    test  = data[60000:]
    return train, test

In [4]:
def display_importances(feature_importance_df):
    cols = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]

    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()

In [5]:
def Submission(Ids, preds):
    file_name = datetime.datetime.today().strftime('%m-%d-%H-%M')
    submission = pd.DataFrame({'building_id' : Ids, 'total_price' : preds})
    if not os.path.isdir('Submission'):
        os.makedirs('Submission')
    submission.to_csv('Submission/' + file_name + '.csv', index= False)

In [12]:
def lgb_model(split_num, train, test, stratified = False, if_one_hot = True):
    

    category_cols = ['building_material','building_type','building_use','parking_way']
    
    if stratified:
        kf = StratifiedKFold(n_splits = split_num, random_state = 42, shuffle = True)
    else :
        kf = KFold(n_splits = split_num, random_state=42, shuffle=True)
    train['total_price_log'] = np.log1p(train['total_price'])
    feature_importance_df = pd.DataFrame()
    oof = np.zeros(len(train))
    predictions = np.zeros(len(test))
    
    param ={
        'n_estimators': 5, 'max_depth' : -1, 'num_leaves' :30,         
        'objective': 'regression',   'metric': 'rmse',   
        'learning_rate': 0.01,      'boosting': 'gbdt',     'min_data_in_leaf': 10,
        'feature_fraction': 0.9,    'bagging_freq':1,       'bagging_fraction': 0.8,     'importance_type': 'gain',
        'lambda_l1': 0.2,  'subsample': .8,   'colsample_bytree': .9, 'device' : 'cpu', 'num_thread' : 8
    }
    features = [i for i in train.columns if i not in ['total_price_log', 'total_price', 'city', 'building_id']]

    for fold_, (trn_idx, val_idx) in enumerate(kf.split(train[features].values,train['total_price_log'].values)):
        trn_data = lgb.Dataset(train.iloc[trn_idx][features], label= train['total_price_log'].iloc[trn_idx])
        val_data = lgb.Dataset(train.iloc[val_idx][features], label= train['total_price_log'].iloc[val_idx])
        
        clf = lgb.train(params= param, train_set= trn_data, valid_sets= [trn_data, val_data], verbose_eval=100, early_stopping_rounds= 10000, categorical_feature=category_cols, feval=hit_score)
        oof[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration = clf.best_iteration)
        predictions += clf.predict(test[features], num_iteration = clf.best_iteration) / kf.n_splits
        
        y   = np.expm1(train['total_price_log'].iloc[val_idx])
        yhat = np.expm1(oof[val_idx])
        Hit_score = np.sum([1 for i in np.abs((y - yhat) / y)  if i <= 0.1 ])
        print('fold {} hit_score : {}'.format(fold_ + 1, round(Hit_score, 4) /len(train.iloc[val_idx]) * 10000))
        print('-'*30)
        fold_importance_df = pd.DataFrame()
        fold_importance_df['feature']    = features
        fold_importance_df['importance'] = np.log1p(clf.feature_importance(importance_type='gain', iteration=clf.best_iteration))
        fold_importance_df['fold']       = fold_ + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    print('CV scrore : {}'.format(sqrt(mean_squared_error(train['total_price_log'], oof))))
    print('-'*30)
    y = np.expm1(train['total_price_log']) 
    yhat = np.expm1(oof)
    Hit_score = np.sum([1 for i in np.abs((y - yhat) / y)  if i <= 0.1 ])
    print('Hit rate : {}'.format(round(Hit_score, 4) /len(train) * 10000))
    
    display_importances(feature_importance_df)
    return predictions, round(Hit_score, 4) /len(train) * 10000, oof


In [9]:
def feature_process(df):
    
    #Impute missing value
    #df['village_income_median'] = df.groupby(['city', 'town'])['village_income_median'].transform(lambda x : x .fillna(x.median())))
    
    #floor
    df.loc[df['txn_floor'].isna(), 'department'] = 1 
    df.loc[df['txn_floor'].notna(), 'department'] = 0
    df.loc[df['txn_floor'].isna(), 'txn_floor'] = df.loc[df['txn_floor'].isna(), 'total_floor'] 
    df['avg_height_floor'] = df['txn_floor'] / df['total_floor'] 
    df['avg_height_floor'].fillna(0, inplace = True)
    
    #location
    df['location_2'] = df.apply(lambda x : int(str(x['town'])), axis=1)
    df['location_2'] = df['location_2'].astype('category')
    df['town'] = df['town'].astype('category')
    
    #date
    df['day_between_txn_complete'] = df['txn_dt'] - df['building_complete_dt']
    df['year_between_txn_complete'] = round(df['day_between_txn_complete'] / 365)
    
    #area
    #df['land*bulid_area'] = df['land_area'] * df ['building_area']
    df['land/bulid_area'] = df['land_area'] / df ['building_area']
    df['house_area'] = df['building_area'] - df['parking_area']
    df['house_rate']  = df['building_area'] / df['house_area']
    
    #parking
    df['miss_parking_area'] = 0
    df['miss_parking_price'] = 0
    df.loc[df['parking_area'].isna(), 'miss_parking_area'] = 1
    df.loc[df['parking_price'].isna(), 'miss_parking_price'] = 1
    df['parking_price_every_area'] = df['parking_price'] / df['parking_area']
    df['parking_way'] = df['parking_way'].astype('category')
    df['parking_area'].fillna(0, inplace =True)
    df['parking_price'].fillna(0, inplace =True)
    
    #population

    
    #building   #building_type = 4 is house
    df['building_type'] = df['building_type'].astype('category')
    df['building_use'] = df['building_use'].astype('category')
    df['building_material'] = df['building_material'].astype('category')
    #df = df.join(pd.get_dummies(df['building_type'], prefix = 'building_type'))
    
    #MIN
    
    MIN_cols = [i for i in df.columns if '_MIN' in i]
    df['min_cat'] = 0
    for col in [i for i in df.columns if 'MIN' in i]:
        df['min_cat'] = df.apply(lambda x : col if x[col] == min(x[MIN_cols]) else x['min_cat'], axis=1)
    le = preprocessing.LabelEncoder()
    df['min_cat'] = le.fit_transform(df['min_cat'])
    
    
    df['MIN_dis_plus'] = reduce(lambda x,y: x + y, [df[i] for i in [i for i in df.columns if '_MIN' in i]])
    df['MIN_dis'] = df.apply(lambda x : x[MIN_cols].min(), axis = 1)
    df['MAX_dis'] = df.apply(lambda x : x[MIN_cols].max(), axis = 1)
    df['Std_dis'] = df.apply(lambda x : x[MIN_cols].std(), axis = 1)
    df['Median_dis'] = df.apply(lambda x : x[MIN_cols].median(), axis = 1)
    df['Diff_dis'] = df['MAX_dis'] - df['MIN_dis']
    
    #10 50 100 250 500 1000 5000 10000
    df['diff_500_50'] = df['N_500'] - df['N_50']
    df['diff_1000_500'] = df['N_1000'] - df['N_500']
    df['diff_5000_1000'] = df['N_5000'] - df['N_1000']
    df['diff_10000_5000'] = df['N_10000'] - df['N_5000']
    df['rate_500_50'] = df['N_500'] / df['N_50']
    df['rate_1000_500'] = df['N_1000'] / df['N_500']
    df['rate_5000_1000'] = df['N_5000'] / df['N_1000']
    df['rate_10000_5000'] = df['N_10000'] / df['N_5000']
    
    
    All_10    = [i for i in df.columns if i.endswith('_10') and 'index' not in i and 'N' not in i]
    All_50    = [i for i in df.columns if i.endswith('_50') and 'index' not in i and 'N' not in i]
    All_100   = [i for i in df.columns if i.endswith('_100') and 'index' not in i and 'N' not in i]
    All_250   = [i for i in df.columns if i.endswith('_250') and 'index' not in i and 'N' not in i]
    All_500   = [i for i in df.columns if i.endswith('_500') and 'index' not in i and 'N' not in i]
    All_1000  = [i for i in df.columns if i.endswith('_1000') and 'index' not in i and 'N' not in i]
    All_5000  = [i for i in df.columns if i.endswith('_5000') and 'index' not in i and 'N' not in i]
    All_10000 = [i for i in df.columns if i.endswith('_10000') and 'index' not in i and 'N' not in i]
    df['All_10'] = reduce(lambda x, y: x + y, [df[i] for i in All_10])
    df['All_50'] = reduce(lambda x, y: x + y, [df[i] for i in All_50])
    df['All_100'] = reduce(lambda x, y: x + y, [df[i] for i in All_100])
    df['All_250'] = reduce(lambda x, y: x + y, [df[i] for i in All_250])
    df['All_500'] = reduce(lambda x, y: x + y, [df[i] for i in All_500])
    df['All_1000'] = reduce(lambda x, y: x + y, [df[i] for i in All_1000])
    df['All_5000'] = reduce(lambda x, y: x + y, [df[i] for i in All_5000])
    df['All_10000'] = reduce(lambda x, y: x + y, [df[i] for i in All_10000])
    
    '''
    for i,j in  zip([All_10, All_50, All_100, All_250, All_500, All_1000, All_5000], [All_50, All_100, All_250, All_500, All_1000, All_5000, All_10000]):
        for order in range(len(i)):
            df[j[order] + '_' + i[order] + '_rate'] = df[j[order]] / df[i[order]]
    '''
    #interection
    inter_cols = ['building_type', 'parking_way', 'building_use', 'building_material']
    for i in range(4):
        for j in range(4):
            if j > i:
                df['inter_btw_' + inter_cols[i] + '_' + inter_cols[j]] = df.apply(lambda x : str(x[inter_cols[i]]) + str(x[inter_cols[j]]), axis=1)
                df['inter_btw_' + inter_cols[i] + '_' + inter_cols[j]] = df['inter_btw_' + inter_cols[i] + '_' + inter_cols[j]].astype('category')
    

    
    #groupby encoding
    
    category_cols  = ['building_material','building_type','building_use','parking_way','location_2']
    numerical_cols = ['building_area', 'land_area', 'day_between_txn_complete', 'txn_dt', 'building_complete_dt', 'house_area', 'house_rate']
    statistics = ['mean', 'median', 'max', 'min']
    for category in category_cols:
        for numerical in numerical_cols:
            for stat in statistics:
                df[numerical + '_' + stat + '_gb_' + category] = df.groupby([category])[numerical].transform(stat)
                df[numerical + '_diff_' + stat + '_gb_' + category] = df[numerical] - df[numerical + '_' + stat + '_gb_' + category]

    category_cols =['town', 'location_2']
    count_cols = ['building_type', 'building_use', 'building_material', 'parking_way']
    for category in category_cols:
        for count_col in count_cols:
            df['size_gb_' + category + '_' + count_col] = df.groupby([category, count_col])[count_col].transform('size')
    
    #Polynomail feature
    
    #useless cols
    df.drop([i for i in df.columns if 'index' in i and np.sum(df[i]) == 60000], axis = 1, inplace = True)
    return df

In [10]:
begin = time.time()
Final_data = feature_process(data)
train = Final_data[:60000]
test = Final_data[60000:]
gc.collect()
(time.time() - begin) / 60 

17.415748504797616

In [13]:
begin = time.time()
avg_hit_rate = 0
prediction_df = pd.DataFrame()
hit_score_list = []
train_num_list = []
city_df_list   = []
oof_list       = []

for city in train['city'].unique():
    temp_train = train[train['city'] == city]
    temp_test  = test[test['city'] == city]
    temp_train.drop(['city'], axis =1, inplace =True)
    temp_test.drop(['city'], axis =1, inplace =True)
    
    target_df = temp_train.groupby(['town']).agg({'building_area' : ['mean', 'median'], 'land_area' : ['mean', 'median'], 'total_price' : ['mean', 'median']}).reset_index()
    target_df.columns = [i[0] + '_' + i[1]  if i[1] != '' else i[0] for i in target_df.columns.tolist()]
    target_df['price_land_rate_median'] = target_df['total_price_median'] / target_df['land_area_median']
    target_df['price_building_rate_median'] = target_df['total_price_median'] / target_df['building_area_median']
    target_df['price_land_rate_mean'] = target_df['total_price_mean'] / target_df['land_area_mean']
    target_df['price_building_rate_mean'] = target_df['total_price_mean'] / target_df['building_area_mean']

    combine_cols = ['town', 'price_land_rate_median', 'price_building_rate_median', 'price_land_rate_mean', 'price_building_rate_mean']
    temp_train = pd.merge(temp_train, target_df[combine_cols], on =['town'], how='left')
    temp_test = pd.merge(temp_test, target_df[combine_cols], on =['town'], how='left')
    
    preds, hit_score, oof = lgb_model(5, temp_train, temp_test, if_one_hot=False)
    temp = pd.DataFrame({'building_id' : temp_test['building_id'], 'total_price' : preds})
    prediction_df = pd.concat([prediction_df, temp], axis=0)
    
    print('City : {}'.format(city))
    print('Train_num: {}'.format(len(temp_train)))
    print('Test_num: {}'.format(len(temp_test)))
    print('-'*1000)
    hit_score_list.append(hit_score)
    train_num_list.append(len(temp_train))
    city_df_list.append(city)
    oof_list.append(oof)
    avg_hit_rate += hit_score / 60000 * len(temp_train)

Result_df = pd.DataFrame({'City' : city_df_list,
                        'Train_num' : train_num_list,
                        'Hit_score' : hit_score_list})

print('Avg hit_score : {}'.format(avg_hit_rate))
print('總共花：{} 分'.format((time.time() - begin) / 60))
print('現在時間 ： {}'.format(datetime.datetime.today().strftime('%m-%d-%H-%M')))
Submission(prediction_df['building_id'], np.expm1(prediction_df['total_price']))

Training until validation scores don't improve for 10000 rounds.


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().